In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cftime

import IPython.display
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
import sys, os, warnings

import cosima_cookbook as cc

In [27]:
db = 'local_cc_test.db'
session = cc.database.create_session(db)

In [14]:
from collections import OrderedDict
exptdict = OrderedDict([
    ('HI-00', # original historical run
     {'model': 'ACCESS-ESM 1.5', 'expt': 'HI-00',
      'n_files': -12, 'itime': '1850-01-01', 'ftime': None}),
    ('HI-02', # Pacific Pacemaker historical run
     {'model': 'ACCESS-ESM 1.5', 'expt': 'HI-02',
      'n_files': -12, 'itime': '1850-01-01', 'ftime': None}),
])

In [24]:
# Add on pre-interpolated WOA13 directories for every run:
session_obs = cc.database.create_session()
for ekey in exptdict.keys():
    e = exptdict[ekey]
    if (ekey.find('025deg') != -1):
        e['WOA13expt'] = '025_KDS50'
        e['WOA13file'] = 'woa13_ts_??_mom025.nc'
    elif (ekey.find('01deg') != -1):
        e['WOA13expt'] = '01'
        e['WOA13file'] = 'woa13_ts_??_mom01.nc'
    else:
        e['WOA13expt'] = '10_KDS50'
        e['WOA13file'] = 'woa13_ts_??_mom10.nc'

In [26]:
cc.querying.get_ncfiles(session_obs,experiment='10_KDS50')

,ncfile,index_time


In [11]:
# Function to extract and load SST and SSS from the models, WOA13
ekeys = ['HI-00', 'HI-02']
for ekey in ekeys:
    e = exptdict[ekey]

    # SST
    # -----

    # Load surface temperature from model
    surface_temp = cc.querying.getvar(e['expt'], 'pot_temp', session,
                            n=e['n_files'], ncfile='ocean_month.nc',
                 start_time=e['itime'], end_time=e['ftime']).isel(st_ocean=0)

    # Extract a year range string and print (for title string):
    e['yearrange'] = "{} to {}".format(
        surface_temp.time.item(0), #.strftime("%Y-%m"),
        surface_temp.time.item(-1), #.strftime("%Y-%m"),
    )
    print(f"{ekey}: {e['yearrange']}")

    # Add SST to dictionary
    e['SST'] = surface_temp.mean('time').load() - 273.15

    # Load WOA13 SST and add to dictionary
    # ignore warning about ambiguous files
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        e['SST_WOA13'] = cc.querying.getvar(e['WOA13expt'], 'temp', session_obs).isel(ZT=0).mean('time').load()

    # Calculate bias and add to dictionary
    SST_diff = e['SST'] - e['SST_WOA13'].values
    e['SST_diff'] = SST_diff.load()

    # SSS
    # -----
    surface_salt = cc.querying.getvar(e['expt'], 'salt', session,
                            n=e['n_files'],ncfile='ocean_month.nc',
                 start_time=e['itime'], end_time=e['ftime']).isel(st_ocean=0)
    e['SSS'] = surface_salt.mean('time').load()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)
        e['SSS_WOA13'] = cc.querying.getvar(e['WOA13expt'], 'salt',session_obs).isel(ZT=0).mean('time').load()
    SSS_diff = e['SSS'] - e['SSS_WOA13'].values
    e['SSS_diff'] = SSS_diff.load()

HI-00: -3375432000000000000 to -2999419200000000000


VariableNotFoundError: No files were found containing 'temp' in the '10_KDS50' experiment